# Jean Lafitte Data Management 
### Check section title for proper use of code

#### Package Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates as mpl_dates
import os
import glob

#### Basic Grouping for Single Well

In [ ]:
df = pd.read_csv('filename', encoding = 'unicode_escape', low_memory=False)
df.columns = df.columns.map(lambda x: x.strip())
df['DateandTime'] = pd.to_datetime(df['DateandTime'])
df = df.groupby(['DateandTime','Well_ID'], as_index=False)['level(surface)'].mean()
df = df.set_index('DateandTime')
#df = df.loc['01/01/2014':'12/31/2020']
df = df.groupby([pd.Grouper(freq="H"),'Well_ID'])['level(surface)'].mean().reset_index()
#df.to_csv ('ChosenWellCombined.csv', mode= 'a',index = False)
print(df[0:3])

#### Use to get count of days per month with water levels over 0

In [ ]:
for filepath in glob.iglob("C:\\Users\\Wally\\Documents\\JL_WELLS\\NEW_ITERATIONS22\*.csv"):
    print(filepath)
    df = pd.read_csv(filepath, encoding = 'unicode_escape', low_memory=False)
    df.columns = df.columns.map(lambda x: x.strip())
    df['DateandTime'] = pd.to_datetime(df['DateandTime'])
    df = df.groupby(['DateandTime','Well_ID'], as_index=False)['level(surface)'].mean()
    df = df.set_index('DateandTime')
    #df = df.loc['01/01/2016':'12/31/2016']
    df = df.groupby([pd.Grouper(freq="M"),'Well_ID'])['level(surface)'].agg(['mean','min','max', (lambda x: (x > 0).sum()/24)]).reset_index()
    df.to_csv ('New2022Data_countWETdays.csv', mode= 'a', index = False)
    print(df)

#### Get local Min & Max values for drain rate based on 12hr scale

In [ ]:
for filepath in glob.iglob("C:\\Users\\Wally\\Documents\\JL_WELLS\\NEW_ITERATIONS22\*.csv"):
    print(filepath)
    df = pd.read_csv(filepath, encoding = 'unicode_escape', low_memory=False)
    df.columns = df.columns.map(lambda x: x.strip())
    df['DateandTime'] = pd.to_datetime(df['DateandTime'])
    df = df.groupby(['DateandTime','Well_ID'], as_index=False)['level(surface)'].mean()
    df = df.set_index('DateandTime')
    #df = df.loc['01/01/2016':'12/31/2020']
    df = df.groupby([pd.Grouper(freq="12H"),'Well_ID'])['level(surface)'].mean().reset_index()
    df.rename(columns ={'level(surface)':'level'}, inplace = True)
    #df.to_csv ('countWETdaysWELL28.csv', mode= 'a', index = False)
    print(df[0:2])
    

      #Find local peaks
    df['min'] = df.level[(df.level.shift(1) > df.level) & (df.level.shift(-1) > df.level)]
    df['max'] = df.level[(df.level.shift(1) < df.level) & (df.level.shift(-1) < df.level)]
    df.to_csv ('16-20allDrainRate.csv', mode= 'a', index = False)
    print (df[0:1])
    
 # Plot results
plt.scatter(df.index, df['min'], c='r')
plt.scatter(df.index, df['max'], c='g')
df.level.plot()